# Shallow MLP


# Preliminaries

Check for GPU

In [1]:
import tensorflow as tf

In [2]:
if tf.test.gpu_device_name() != '/device:GPU:0':
  print("No GPU found")
else:
  print("GPU ready: {}".format(tf.test.gpu_device_name()))

No GPU found


Import preprocessing helper functions

In [3]:
import sys
sys.path.append("../../helper-modules")
from preprocessing_utils import read_in_data, preprocess

Read in the data to df_train, df_val and df_test

In [4]:
df_train, df_val, df_test = read_in_data()

Create X_train, y_train, X_val, y_val, X_test, y_test

In [5]:
(X_train, y_train), (X_val, y_val), (X_test, y_test) = preprocess("MLP", df_train, df_test, df_val)

# Model Building

Imports

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from itertools import product
import time
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical

Enable seed setting for improved reproducibility

In [7]:
import os
import random
import numpy as np

In [8]:
def set_seed(seed=42):
  SEED=seed

  # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
  os.environ['PYTHONHASHSEED']=str(SEED)

  # 2. Set `python` built-in pseudo-random generator at a fixed value
  random.seed(SEED)

  # 3. Set `numpy` pseudo-random generator at a fixed value
  np.random.seed(SEED)

  # 4. Set the `tensorflow` pseudo-random generator at a fixed value
  tf.random.set_seed(SEED)

Extract number of classes (10 classes)

In [9]:
NUM_CLASSES = df_test["label"].nunique() 

Model design

In [10]:
def compile_model(hidden_layers_sizes=(1024,), act_function="relu", dropout_rate=0.1, l_rate=0.001, reg='l2', opt_algo='adam'):
    """
    Compiles a shallow MLP model with one hidden layer with the given hyperparameters.
    
    Hyperparameter options:
    
    hidden_layers_sizes : tuple with the number of neurons in the hidden layers
    act_function : options include "relu", "sigmoid" and "tanh"
    dropout_rate : a float with the rate of dropout to be applied to the fully connected layer
    l_rate : a float with the learning rate
    reg : string with the regularization type e.g. "l2"
    opt_algo : the optimization algorithm, "adam" or "sgd"
    """  
    
    model = Sequential()

    # Add hidden layers with dropout to prevent overfitting
    for i, h in enumerate(hidden_layers_sizes):
      if i == 0:
        model.add(Dense(h, activation=act_function, input_shape=(1480,)))
      else: 
        model.add(Dense(h, activation=act_function))
      model.add(Dropout(dropout_rate))
    
    # LAST LAYER IS THE CLASSIFIER, THUS 12 POSSIBLE CLASSES
    model.add(Dense(NUM_CLASSES, activation='softmax', kernel_regularizer=reg))

    # Optimization algorithm
    if opt_algo == 'adam':
      opt = tf.keras.optimizers.Adam(learning_rate=l_rate)
    elif opt_algo == 'sgd':
      opt = tf.keras.optimizers.SGD(learning_rate=l_rate)

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

# Experiment: Grid Search

Defining grid search hyperparameter options

In [11]:
hidden_layers_sizes = [(1,), (5,), (20,), (81,), (322,), (1288,)]
l_rates = [0.01, 0.005, 0.001, 0.0005, 0.0001]
dropout_rates = [0.05, 0.1, 0.2, 0.5]
act_functions=["relu"]
regs = ['l2']
opt_algos = ['adam']

all_hparams = [hidden_layers_sizes, l_rates, dropout_rates, act_functions, regs, opt_algos]

Conduct the grid search

In [12]:
def grid_search():
  """
  For each `hidden_layers_sizes` option (corresponding to a given number of parameters), conduct a grid search
  over the hyperparameter options, writing the validation results and model details for each model to a file.
  
  All trained models are saved as h5 files. 
  """
 
  best_model = None
  best_accuracy = 0
  best_history = None

  EPOCHS = 50
  BATCH_SIZE = 32
  STOPPING_PATIENCE = 5

  STARTING_POINT = 1

  with open('../training-results/ExperimentLogs_Shallow-MLP.csv', 'w') as log_file:
      log_file.write("ModelNumber;Timestamp;NumEpochs;ValAccuracy;NumParams;HiddenLayerSizes;LearningRate;DropoutRate;ActivationFunction;Regularization;OptimizationAlgorithm\n")

  for model_num, hparam_set in enumerate(list(product(*all_hparams))[0:], start=STARTING_POINT):
    # Extract hyperparams for current model from grid search grid
    hidden_layers_sizes_tup, l_rate, dropout_rate, act_function, reg, opt_algo = hparam_set

    # Define hyperparameters for the current model 
    hparams = {
        "hidden_layers_sizes" : hidden_layers_sizes_tup,
        "act_function" : act_function, 
        "dropout_rate" : dropout_rate, 
        "l_rate": l_rate,
        "reg" : reg,
        "opt_algo" : opt_algo
    }

    # Compile model and count number of parameters
    model = compile_model(**hparams)
    num_params = model.count_params()

    model_str = f"{hidden_layers_sizes_tup}; {l_rate}; {dropout_rate}; {act_function}; {reg}; {opt_algo}"
    print(f"Training model {model_num} with {num_params} params - {model_str}")

    # Prevent overfitting with early stopping
    early_stop = EarlyStopping(monitor='val_accuracy', patience=STOPPING_PATIENCE)
    # For saving best val accuracy model as an h5 (for test predictions later)
    model_check = ModelCheckpoint(f"../trained-models/Shallow-MLP_{model_num}.h5", monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)

    # Set seed for reproducibility
    set_seed(42)

    # Train model
    # Note: one reason val accuracy might be higher than train accuracy during training is because dropout affects training but not validation
    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), callbacks=[early_stop, model_check])
    n_epochs = len(history.history['loss'])
                   
    # Time stamp when model finished training
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    # Extract best validation accuracy at inex 1 (index 0 has the loss)
    val_accuracy = max(history.history['val_accuracy'])
    print(f"Best val accuracy: {val_accuracy}")

    # Store results in log file
    with open('../training-results/ExperimentLogs_Shallow-MLP.csv', 'a') as log_file:
      log_file.write(f"{model_num};{timestamp};{n_epochs};{val_accuracy};{num_params};{model_str}\n")

    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      best_model = model
      best_history = history

  # Return the model with the highest validation accuracy
  return best_model, best_history


In [ ]:
best_model, best_history = grid_search()

Training model 1 with 1507 params - (1,); 0.01; 0.05; relu; l2; adam
Epoch 1/50
2600/2600 [==============================] - 2s 682us/step - loss: 2.3153 - accuracy: 0.1736 - val_loss: 1.9921 - val_accuracy: 0.2044
Epoch 2/50
2600/2600 [==============================] - 1s 490us/step - loss: 2.1002 - accuracy: 0.2159 - val_loss: 1.9877 - val_accuracy: 0.2319
Epoch 3/50
2600/2600 [==============================] - 1s 520us/step - loss: 2.0945 - accuracy: 0.2452 - val_loss: 1.9647 - val_accuracy: 0.2742
Epoch 4/50
2600/2600 [==============================] - 1s 467us/step - loss: 2.0997 - accuracy: 0.2395 - val_loss: 2.0892 - val_accuracy: 0.1677
Epoch 5/50
2600/2600 [==============================] - 1s 460us/step - loss: 2.1030 - accuracy: 0.2339 - val_loss: 2.1115 - val_accuracy: 0.2012
Epoch 6/50
2600/2600 [==============================] - 1s 468us/step - loss: 2.1209 - accuracy: 0.2303 - val_loss: 1.9689 - val_accuracy: 0.2375
Epoch 7/50
2600/2600 [==============================] -

Epoch 1/50
2600/2600 [==============================] - 2s 548us/step - loss: 2.4701 - accuracy: 0.1292 - val_loss: 2.2179 - val_accuracy: 0.1850
Epoch 2/50
2600/2600 [==============================] - 1s 478us/step - loss: 2.2052 - accuracy: 0.1986 - val_loss: 2.0817 - val_accuracy: 0.2217
Epoch 3/50
2600/2600 [==============================] - 1s 476us/step - loss: 2.1158 - accuracy: 0.2152 - val_loss: 2.0327 - val_accuracy: 0.2354
Epoch 4/50
2600/2600 [==============================] - 1s 476us/step - loss: 2.0774 - accuracy: 0.2247 - val_loss: 2.0055 - val_accuracy: 0.2346
Epoch 5/50
2600/2600 [==============================] - 1s 475us/step - loss: 2.0634 - accuracy: 0.2229 - val_loss: 1.9899 - val_accuracy: 0.2330
Epoch 6/50
2600/2600 [==============================] - 1s 473us/step - loss: 2.0636 - accuracy: 0.2261 - val_loss: 1.9881 - val_accuracy: 0.2450
Epoch 7/50
2600/2600 [==============================] - 1s 466us/step - loss: 2.0599 - accuracy: 0.2324 - val_loss: 1.9839 -

2600/2600 [==============================] - 1s 391us/step - loss: 2.0582 - accuracy: 0.2373 - val_loss: 1.9882 - val_accuracy: 0.2393
Epoch 14/50
2600/2600 [==============================] - 1s 394us/step - loss: 2.0604 - accuracy: 0.2309 - val_loss: 1.9836 - val_accuracy: 0.2532
Epoch 15/50
2600/2600 [==============================] - 1s 392us/step - loss: 2.0570 - accuracy: 0.2358 - val_loss: 1.9800 - val_accuracy: 0.2656
Epoch 16/50
2600/2600 [==============================] - 1s 391us/step - loss: 2.0569 - accuracy: 0.2352 - val_loss: 1.9804 - val_accuracy: 0.2430
Epoch 17/50
2600/2600 [==============================] - 1s 393us/step - loss: 2.0472 - accuracy: 0.2381 - val_loss: 1.9779 - val_accuracy: 0.2695
Epoch 18/50
2600/2600 [==============================] - 1s 393us/step - loss: 2.0536 - accuracy: 0.2353 - val_loss: 1.9812 - val_accuracy: 0.2724
Epoch 19/50
2600/2600 [==============================] - 1s 395us/step - loss: 2.0598 - accuracy: 0.2491 - val_loss: 1.9793 - val_

2600/2600 [==============================] - 1s 393us/step - loss: 2.1066 - accuracy: 0.1869 - val_loss: 2.0502 - val_accuracy: 0.2050
Epoch 16/50
2600/2600 [==============================] - 1s 395us/step - loss: 2.0981 - accuracy: 0.1921 - val_loss: 2.0469 - val_accuracy: 0.1917
Epoch 17/50
2600/2600 [==============================] - 1s 395us/step - loss: 2.0871 - accuracy: 0.1867 - val_loss: 2.0420 - val_accuracy: 0.1911
Epoch 18/50
2600/2600 [==============================] - 1s 395us/step - loss: 2.0855 - accuracy: 0.1922 - val_loss: 2.0268 - val_accuracy: 0.1995
Epoch 19/50
2600/2600 [==============================] - 1s 394us/step - loss: 2.0869 - accuracy: 0.1870 - val_loss: 2.0235 - val_accuracy: 0.1943
Epoch 20/50
2600/2600 [==============================] - 1s 394us/step - loss: 2.0678 - accuracy: 0.1904 - val_loss: 2.0188 - val_accuracy: 0.1924
Best val accuracy: 0.2049519270658493
Training model 18 with 1507 params - (1,); 0.0001; 0.1; relu; l2; adam
Epoch 1/50
2600/2600 

2600/2600 [==============================] - 1s 423us/step - loss: 1.4581 - accuracy: 0.4794 - val_loss: 1.4319 - val_accuracy: 0.4528
Epoch 6/50
2600/2600 [==============================] - 1s 424us/step - loss: 1.4445 - accuracy: 0.4878 - val_loss: 1.3093 - val_accuracy: 0.5234
Epoch 7/50
2600/2600 [==============================] - 1s 435us/step - loss: 1.4412 - accuracy: 0.4916 - val_loss: 1.4090 - val_accuracy: 0.4833
Epoch 8/50
2600/2600 [==============================] - 1s 437us/step - loss: 1.4609 - accuracy: 0.4845 - val_loss: 1.3388 - val_accuracy: 0.4901
Epoch 9/50
2600/2600 [==============================] - 1s 450us/step - loss: 1.4526 - accuracy: 0.4899 - val_loss: 1.7303 - val_accuracy: 0.4313
Epoch 10/50
2600/2600 [==============================] - 1s 429us/step - loss: 1.4878 - accuracy: 0.4875 - val_loss: 1.3570 - val_accuracy: 0.5241
Epoch 11/50
2600/2600 [==============================] - 1s 422us/step - loss: 1.4705 - accuracy: 0.4881 - val_loss: 1.5587 - val_accu

...

Epoch 3/50
2600/2600 [==============================] - 1070s 412ms/step - loss: 6.4304 - accuracy: 0.5470 - val_loss: 7.5208 - val_accuracy: 0.6160
Epoch 4/50
2600/2600 [==============================] - 3s 1ms/step - loss: 8.1966 - accuracy: 0.5515 - val_loss: 9.7225 - val_accuracy: 0.5710
Epoch 5/50
2600/2600 [==============================] - 917s 353ms/step - loss: 10.2287 - accuracy: 0.5572 - val_loss: 11.6922 - val_accuracy: 0.5746
Epoch 6/50
2600/2600 [==============================] - 1005s 387ms/step - loss: 12.5063 - accuracy: 0.5597 - val_loss: 14.0785 - val_accuracy: 0.6075
Epoch 7/50
2600/2600 [==============================] - 3s 1ms/step - loss: 14.8105 - accuracy: 0.5629 - val_loss: 17.0720 - val_accuracy: 0.6129
Epoch 8/50
2600/2600 [==============================] - 1113s 428ms/step - loss: 17.6875 - accuracy: 0.5645 - val_loss: 19.0771 - val_accuracy: 0.6089
Best val accuracy: 0.6159615516662598
Training model 83 with 481081 params - (322,); 0.01; 0.2; relu; l2; ada

Epoch 2/50
2600/2600 [==============================] - 3s 1ms/step - loss: 2.8634 - accuracy: 0.6383 - val_loss: 3.4980 - val_accuracy: 0.6871
Epoch 3/50
2600/2600 [==============================] - 3s 1ms/step - loss: 3.7494 - accuracy: 0.6755 - val_loss: 4.5205 - val_accuracy: 0.6910
Epoch 4/50
2600/2600 [==============================] - 3s 1ms/step - loss: 4.9047 - accuracy: 0.6908 - val_loss: 5.8887 - val_accuracy: 0.6966
Epoch 5/50
2600/2600 [==============================] - 3s 1ms/step - loss: 6.2241 - accuracy: 0.7025 - val_loss: 7.3379 - val_accuracy: 0.7385
Epoch 6/50
2600/2600 [==============================] - 3s 1ms/step - loss: 7.9065 - accuracy: 0.7120 - val_loss: 9.2889 - val_accuracy: 0.7088
Epoch 7/50
2600/2600 [==============================] - 3s 1ms/step - loss: 9.7806 - accuracy: 0.7237 - val_loss: 11.1979 - val_accuracy: 0.7294
Epoch 8/50
2600/2600 [==============================] - 3s 1ms/step - loss: 11.6432 - accuracy: 0.7178 - val_loss: 12.8902 - val_accura

2600/2600 [==============================] - 3s 1ms/step - loss: 5.1019 - accuracy: 0.8256 - val_loss: 5.7993 - val_accuracy: 0.7670
Epoch 10/50
2600/2600 [==============================] - 3s 1ms/step - loss: 5.7164 - accuracy: 0.8365 - val_loss: 6.4159 - val_accuracy: 0.7745
Epoch 11/50
2600/2600 [==============================] - 3s 1ms/step - loss: 6.3387 - accuracy: 0.8426 - val_loss: 7.0799 - val_accuracy: 0.7632
Epoch 12/50
2600/2600 [==============================] - 3s 1ms/step - loss: 6.9653 - accuracy: 0.8492 - val_loss: 7.7522 - val_accuracy: 0.7773
Epoch 13/50
2600/2600 [==============================] - 3s 1ms/step - loss: 7.6184 - accuracy: 0.8581 - val_loss: 8.4416 - val_accuracy: 0.7740
Epoch 14/50
2600/2600 [==============================] - 3s 1ms/step - loss: 8.2967 - accuracy: 0.8663 - val_loss: 9.1162 - val_accuracy: 0.7765
Epoch 15/50
2600/2600 [==============================] - 3s 1ms/step - loss: 8.9298 - accuracy: 0.8736 - val_loss: 9.7773 - val_accuracy: 0.78

2600/2600 [==============================] - 3s 1ms/step - loss: 3.8653 - accuracy: 0.8514 - val_loss: 4.3681 - val_accuracy: 0.7747
Epoch 13/50
2600/2600 [==============================] - 3s 1ms/step - loss: 4.1635 - accuracy: 0.8590 - val_loss: 4.6836 - val_accuracy: 0.7756
Epoch 14/50
2600/2600 [==============================] - 3s 1ms/step - loss: 4.4619 - accuracy: 0.8674 - val_loss: 5.0425 - val_accuracy: 0.7664
Epoch 15/50
2600/2600 [==============================] - 3s 1ms/step - loss: 4.7661 - accuracy: 0.8748 - val_loss: 5.3505 - val_accuracy: 0.7767
Epoch 16/50
2600/2600 [==============================] - 3s 1ms/step - loss: 5.0716 - accuracy: 0.8835 - val_loss: 5.7199 - val_accuracy: 0.7794
Epoch 17/50
2600/2600 [==============================] - 3s 1ms/step - loss: 5.3748 - accuracy: 0.8883 - val_loss: 6.0117 - val_accuracy: 0.7832
Epoch 18/50
2600/2600 [==============================] - 3s 1ms/step - loss: 5.6666 - accuracy: 0.8984 - val_loss: 6.3508 - val_accuracy: 0.77

2600/2600 [==============================] - 3s 1ms/step - loss: 1.3026 - accuracy: 0.7234 - val_loss: 1.3795 - val_accuracy: 0.7032
Epoch 6/50
2600/2600 [==============================] - 3s 1ms/step - loss: 1.3119 - accuracy: 0.7363 - val_loss: 1.3958 - val_accuracy: 0.7175
Epoch 7/50
2600/2600 [==============================] - 3s 1ms/step - loss: 1.3284 - accuracy: 0.7456 - val_loss: 1.4436 - val_accuracy: 0.7150
Epoch 8/50
2600/2600 [==============================] - 3s 1ms/step - loss: 1.3560 - accuracy: 0.7542 - val_loss: 1.4814 - val_accuracy: 0.7152
Epoch 9/50
2600/2600 [==============================] - 4s 1ms/step - loss: 1.3837 - accuracy: 0.7607 - val_loss: 1.5395 - val_accuracy: 0.7134
Epoch 10/50
2600/2600 [==============================] - 6s 2ms/step - loss: 1.4115 - accuracy: 0.7674 - val_loss: 1.5633 - val_accuracy: 0.7233
Epoch 11/50
2600/2600 [==============================] - 11s 4ms/step - loss: 1.4464 - accuracy: 0.7745 - val_loss: 1.6371 - val_accuracy: 0.7133


In [ ]:
df_train.shape


In [16]:
df_test.shape

(30000, 1481)

In [17]:
df_val.shape

(24000, 1481)